In [2]:
import torch.nn as nn
import torch

import sys, os
import random
import numpy as np
from shutil import copy
import matplotlib.pyplot as plt
from copy import deepcopy

from omegaconf import OmegaConf
import shutil
import pickle
import random
from tqdm import tqdm

from torchvision.datasets.folder import ImageFolder
from torch.utils.data import DataLoader
# from skimage.filters import threshold_local, gaussian

from datetime import datetime


In [3]:
# run_path = '/home/harishbabu/projects/PIPNet/runs/010-CUB-27-imgnet_OOD_cnext26_img=224_nprotos=20'
# run_path = '/home/harishbabu/projects/PIPNet/runs/031-CUB-18-imgnet_cnext26_img=224_nprotos=20_orth-on-rel'
# run_path = '/home/harishbabu/projects/PIPNet/runs/032-CUB-18-imgnet_cnext26_img=224_nprotos=20_orth-on-rel'

# run_path = '/home/harishbabu/projects/PIPNet/runs/035-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel'

# run_path = '/home/harishbabu/projects/PIPNet/runs/043-035_clone-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel'
# run_path = "/home/harishbabu/projects/PIPNet/runs/036-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel_uniformity"
# run_path = "/home/harishbabu/projects/PIPNet/runs/041-035_clone-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel"
# run_path = "/home/harishbabu/projects/PIPNet/runs/042-035_clone-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel"

# run_path = "/home/harishbabu/projects/PIPNet/runs/044-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20-or-4per-desc_orth-on-rel"

# run_path = "/home/harishbabu/projects/PIPNet/runs/046-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=10per-desc_orth-on-rel"
# run_path = "/home/harishbabu/projects/PIPNet/runs/047-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=5per-desc_tanh-desc"
# run_path = "/home/harishbabu/projects/PIPNet/runs/048-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=5per-desc_tanh-desc_unit-sphere"
# run_path = "/home/harishbabu/projects/PIPNet/runs/051-CUB-18-imgnet_cnext26_img=224_nprotos=4per-desc_tanh-desc_unit-sphere_AW=5-TW=2-UW=2-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/052-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=4per-desc_tanh-desc_unit-sphere_AW=5-TW=2-UW=2-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/055-CUB-18_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-softmax_AW=3-TW=2-UW=3-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/056-CUB-18-imgnet_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-softmax_AW=3-TW=2-UW=3-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/057-CUB-18-imgnet_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/058-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2"

# with unit sphere
# run_path = "/home/harishbabu/projects/PIPNet/runs/059-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_finetune=5_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# unit sphere with softmax
# run_path = "/home/harishbabu/projects/PIPNet/runs/065-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_finetune=5_no-meanpool_with-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# original hpipnet with 20 protos per node no KO, no OOD, no tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/062-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=20_no-KO_no-OOD"

# original hpipnet with 20 protos per node no KO, no OOD, WITH tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/063-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=20_no-KO_no-OOD_tanh-desc"

# with unit sphere but no AL+UNI
# run_path = "/home/harishbabu/projects/PIPNet/runs/066-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_finetune=5_no-meanpool_no-softmax_no-align_no-uni_AW=3-TW=2-UW=3-CW=2_batch=20"

# with unit sphere, protopool, with softmax, no tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/067-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# with unit sphere, protopool, with softmax, no tanh-desc, INCORRECT
# run_path = "/home/harishbabu/projects/PIPNet/runs/067-incorrect-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# with unit sphere, protopool, no softmax, no tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/068-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# 071 with bias
# run_path = "/home/harishbabu/projects/PIPNet/runs/071-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_with-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 072 gumbel softmax
# run_path = "/home/harishbabu/projects/PIPNet/runs/072-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-gumbel-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 073 gumbel softmax, tau-1.0
# run_path = "/home/harishbabu/projects/PIPNet/runs/073-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-gumbel-softmax-tau=1_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 075 068 with focal loss
# run_path = "/home/harishbabu/projects/PIPNet/runs/075-068-with-focal_CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_no-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 076 cs followed by softmax. Uses align_pf along with align+uni
# run_path = "/home/harishbabu/projects/PIPNet/runs/076_CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_align-pf-during-training_no-meanpool_no-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2-APW=5_batch=20"

# 074 multiply_cs_softmax
# run_path = "/home/harishbabu/projects/PIPNet/runs/074-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_multi-cs-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 077 unit sphere protopool with cosin no softmax constant 20 protos per node
# run_path = "/home/harishbabu/projects/PIPNet/runs/077_CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=20-sphere-protopool_finetune=5_align-pf-during-training_no-meanpool_no-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 082 unit sphere cs followed by softmax with minmazimize loss
# run_path = "/home/harishbabu/projects/PIPNet/runs/082-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 083 unit sphere cs followed by softmax with minmazimize loss
# run_path = "/home/harishbabu/projects/PIPNet/runs/083-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_no-align_no-uni_no-mm-loss_batch=48"

# 085 unit sphere cs followed by softmax-with-tau with minmazimize loss
# run_path = "/home/harishbabu/projects/PIPNet/runs/085-notebook-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=12"

# 091 basic gaussian multiplier on stage 4
# run_path = "/home/harishbabu/projects/PIPNet/runs/091-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_latent-dim=256_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 092 basic gaussian multiplier on stage 3, 4
# run_path = "/home/harishbabu/projects/PIPNet/runs/092-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=3,4|1.0|50_img=224_latent-dim=256_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 093 128 dim linear
# run_path = "/home/harishbabu/projects/PIPNet/runs/093-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_latent-dim=128_nprotos=20_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=1-CW=2_mm-loss_batch=48"

# 094 128 dim linear
# run_path = "/home/harishbabu/projects/PIPNet/runs/094-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_latent-dim=128_nprotos=20_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=1-CW=2_mm-loss_batch=48"

# 095 ablation 091 without AL+UNI
# run_path = "/home/harishbabu/projects/PIPNet/runs/095-091-woALUNI-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_no-AL_no-UNI_mm-loss_batch=48"

# 096 ablation 091 without AL+UNI
# run_path = "/home/harishbabu/projects/PIPNet/runs/096-091-wfocal-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 097 - 091 with bg
# run_path = "/home/harishbabu/projects/PIPNet/runs/097-091-wbg-CUB-18_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 0100 cub29 with 20 per node
# run_path = "/home/harishbabu/projects/PIPNet/runs/100_CUB-29-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=20_unit-sphere-protopool_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 0101 baseline with 4 per desc per node
# run_path = "/home/harishbabu/projects/PIPNet/runs/101-baseline-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_no-KO_no-OOD"

# 0103 091 with 20 per node
# run_path = "/home/harishbabu/projects/PIPNet/runs/103-091-wProtoPool20PerNode_CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=20_unit-sphere-protopool_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_batch=48"

# 0108 LOU3
# run_path = "/home/harishbabu/projects/PIPNet/runs/108-wProtoPool20PerNode_LOU3_CUB-18-imgnet-bg_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=20_unit-sphere-protopool_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_batch=48"

# 154 pruning CUB-18
# run_path = "/home/harishbabu/projects/PIPNet/runs/154-PruningNaiveHPIPNet_cnext13_CUB-18-imgnet-224_with-equalize-aug_img=224_nprotos=20"

# 155 pruning CUB-190
# run_path = "/home/harishbabu/projects/PIPNet/runs/155-PruningNaiveHPIPNet_cnext13_CUB-190-imgnet-224_with-equalize-aug_img=224_nprotos=20"

# 153 pruning CUB-29
# run_path = "/home/harishbabu/projects/PIPNet/runs/153-PruningNaiveHPIPNet_cnext13_CUB-29-imgnet-224_with-equalize-aug_img=224_nprotos=20"

# 156 pruning CUB-18 LOU3
# run_path = "/home/harishbabu/projects/PIPNet/runs/156-PruningNaiveHPIPNet_LOU3_cnext13_CUB-18-imgnet-224_with-equalize-aug_img=224_nprotos=20"

# 157 pruning CUB-29 LOU3
# run_path = "/home/harishbabu/projects/PIPNet/runs/157-PruningNaiveHPIPNet_LOU3_cnext13_CUB-29-imgnet-224_with-equalize-aug_img=224_nprotos=20"

# run_path = "/home/harishbabu/projects/PIPNet/runs/178-PruningNaiveHPIPNetMaskL1=0.5MaskTrainExtra=15epsEps=60TanhDesc=0.05MinCont=0.1_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "/home/harishbabu/projects/PIPNet/runs/179-PruningNaiveHPIPNetMaskL1=0.5MaskTrainExtra=15epsEps=60TanhDesc=0.05MinCont=0.1_cnext26_CUB-190-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "/home/harishbabu/projects/PIPNet/runs/180-PruningNaiveHPIPNetMaskL1=0.5MaskTrainExtra=15epsEps=60TanhDesc=2.0MinCont=0.1_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "/home/harishbabu/projects/PIPNet/runs/181-PruningNaiveHPIPNetMaskL1=0.5MaskTrainExtra=15epsEps=60TanhDesc=2.0MinCont=0.1_cnext26_CUB-190-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "/home/harishbabu/projects/PIPNet/runs/182-PruningNaiveHPIPNetMaskL1=0.5MaskTrainExtra=15epsEps=60TanhDesc=0.1MinCont=0.1_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "/home/harishbabu/projects/PIPNet/runs/183-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=15epsEps=60TanhDesc=0.05MinCont=0.1_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "/home/harishbabu/projects/PIPNet/runs/185-LOUSet1-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=15epsEps=60TanhDesc=0.05MinCont=0.1_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "/home/harishbabu/projects/PIPNet/runs/191-LOUSet2-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "/home/harishbabu/projects/PIPNet/runs/192-PruningNaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0NoTanhDescMinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "/home/harishbabu/projects/PIPNet/runs/193-LOUSet3-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "/home/harishbabu/projects/PIPNet/runs/195-LOUSet3-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pcOr1pd"
# ckpt_file_name = 'net_trained_last'

# run_path = "/home/harishbabu/projects/PIPNet/runs/199-rerun-LOUSet3-PruningBF=1.1NaiveHPIPNetOODEnt=-0.2MaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pc"
# ckpt_file_name = 'net_trained_90'

# run_path = "/home/harishbabu/projects/PIPNet/runs/197-LOUSet3-PruningBF=1.1NaiveHPIPNetOODEnt=-0.2MaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pcOr1pd"
# ckpt_file_name = 'net_trained_90'

# run_path = "/home/harishbabu/projects/PIPNet/runs/200-rerun-LOUSet1-PruningBF=1.1NaiveHPIPNetOODEnt=-0.2MaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pc"
# ckpt_file_name = 'net_trained_last'

run_path = "/home/harishbabu/projects/PIPNet/runs/201-LOUSet1-PruningBF=1.1NaiveHPIPNetOODEnt=-0.2MaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pcOr1pd"
ckpt_file_name = 'net_trained_90'

# try:
#     sys.path.remove('/home/harishbabu/projects/PIPNet')
# except:
#     pass
# sys.path.insert(0, os.path.join(run_path, 'source_clone'))

print(run_path)

/home/harishbabu/projects/PIPNet/runs/201-LOUSet1-PruningBF=1.1NaiveHPIPNetOODEnt=-0.2MaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pcOr1pd


In [4]:
from pipnet.pipnet import PIPNet, get_network
from util.log import Log
from util.args import get_args, save_args, get_optimizer_nn
from util.data import get_dataloaders
from util.func import init_weights_xavier
from pipnet.train import train_pipnet, test_pipnet
# from pipnet.test import eval_pipnet, get_thresholds, eval_ood
from util.eval_cub_csv import eval_prototypes_cub_parts_csv, get_topk_cub, get_proto_patches_cub
from util.vis_pipnet import visualize, visualize_topk
from util.visualize_prediction import vis_pred, vis_pred_experiments
from util.node import Node
from util.phylo_utils import construct_phylo_tree, construct_discretized_phylo_tree
from util.func import get_patch_size
from util.data import ModifiedLabelLoader

Heatmaps showing where a prototype is found will not be generated because OpenCV is not installed.


In [5]:
# from ete3 import Tree

# # Example tree
# tree = Tree("(A,(B,(P,(C1,C2,C3[&&NHX:name=C3]))));", format=1)

# # Print the tree
# print(tree.get_ascii(show_internal=True))

# # List of node names to leave out
# leave_out_classes = ['C3']

# def replace_node(tree):
#     # Iterate through all nodes using a tree traversal
#     for node in tree.traverse("levelorder"):
#         # Check if the node has children (i.e., is not a leaf)
#         if not node.is_leaf():
#             # Check each child to see if it should be replaced
#             for child in node.children:
#                 # If child is a leaf and its name is in leave_out_classes, replace the parent node with the child
#                 if child.is_leaf() and child.name in leave_out_classes:
#                     # Replace the parent node with this child by connecting all other children of the parent
#                     # to the grandparent, and then add the selected child to the grandparent
#                     grandparent = node.up
#                     if grandparent: # If the node to be replaced is not the root
#                         grandparent.add_child(child)
#                         node.detach()
#                     else: # If the node to be replaced is the root
#                         # tree = child
#                         pass
#                     break # Break the loop after replacement

# # Call the function with the tree
# replace_node(tree)

# # Print the modified tree
# print(tree.get_ascii(show_internal=True))


In [6]:
# from ete3 import Tree

# # Example tree
# tree = Tree("(A,(B,(P,(C1,C2,C3[&&NHX:name=C3]))));", format=1)

# # Print the tree
# print(tree.get_ascii(show_internal=True))

# # List of node names to leave out
# leave_out_classes = ['C3']

# def replace_node(tree):
#     for node in tree.traverse("levelorder"):
#         # Check if the node has children (i.e., is not a leaf)
#         if not node.is_leaf():
#             for child in node.children:
#                 # If child is a leaf and its name is in leave_out_classes
#                 if child.is_leaf() and child.name in leave_out_classes:
#                     grandparent = node.up
#                     if grandparent:
#                         # # Find the index of the node to be replaced in the grandparent's children list
#                         # index = grandparent.children.index(node)
#                         # # Detach the node from the grandparent
#                         # node.detach()
#                         # # Insert the child at the specific index where the parent node was located
#                         # grandparent.add_child(child, position=index)

#                         node_idx = grandparent.children.index(node)
#                         idx_to_parent = {grandparent.children.index(parent_node): parent_node for parent_node in grandparent.children}

#                         # print(idx_to_parent)
#                         # print(node_idx)

#                         # print(grandparent.children)
                        
#                         for parent_node in grandparent.get_children():
#                             parent_node.detach()

#                         # print(grandparent.children)

#                         # print(len(idx_to_parent))
    
#                         for idx in range(len(idx_to_parent)):
#                             if idx == node_idx:
#                                 grandparent.add_child(child)
#                             else:
#                                 grandparent.add_child(idx_to_parent[idx])
                        
                    
#                     else:
#                         # Handling the root node replacement case might require setting a new root
#                         # This case needs special handling depending on your tree structure
#                         pass
#                     break # Break the loop after replacement

# # Call the function with the tree
# replace_node(tree)

# # Print the modified tree
# print(tree.get_ascii(show_internal=True))


# Save leave_out_classes to a variable

In [7]:
# from util.phylogeny import PhylogenyCUB
# from ete3 import Tree, PhyloTree
# format_ = 1

# class LOUPhylogenyCUB(PhylogenyCUB):

#     # Phylogeny class for CUB dataset
#     def __init__(self, filePath, leave_out_classes, node_ids=None, verbose=False):
#         self.node_ids = node_ids
#         self.treeFileNameAndPath = filePath
#         self.total_distance = -1 # -1 means we never calculated it before.

#         self.distance_matrix = {}
#         self.species_groups_within_relative_distance = {}

#         self.get_tree(self.treeFileNameAndPath, leave_out_classes)
#         self.get_total_distance()

#     def get_tree(self, treeFileNameAndPath, leave_out_classes):
#         self.tree = PhyloTree(treeFileNameAndPath, format=format_)
#         self.replace_node(self.tree, leave_out_classes)
#         # setting a dummy name to the internal nodes if it is unnamed
#         for i, node in enumerate(self.tree.traverse("postorder")):
#             if not len(node.name) > 0:
#                 node.name = "temp_" + str(i)

#     def replace_node(self, tree, leave_out_classes):
#         for node in tree.traverse("levelorder"):
#             if not node.is_leaf():
#                 for child in node.children:
#                     if child.is_leaf() and child.name in leave_out_classes:
#                         grandparent = node.up
#                         if grandparent:  # If the node to be replaced is not the root
#                             grandparent.add_child(child)
#                             node.detach()
#                         else:  # This code does not handle replacing the root because it requires redefining the tree object
#                             raise Exception('Attempt to replace root node')
#                         break  # Stop checking other children since replacement is done

args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
args = pickle.load(args_file)

# if args.phylo_config:
#     phylo_config = OmegaConf.load(args.phylo_config)

if ('leave_out_classes' in args) and (args.leave_out_classes != ''):
        with open(args.leave_out_classes, 'r') as file:
            leave_out_classes = [line.strip() for line in file]

# tree = LOUPhylogenyCUB(phylo_config.phylogeny_path, leave_out_classes).tree

# tree.write(outfile='tree_LOU.newick', format=1)
    
# print(tree.get_ascii(show_internal=True))

# Define tree

In [8]:
args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
args = pickle.load(args_file)

if args.phylo_config:
    phylo_config = OmegaConf.load(args.phylo_config)

if args.phylo_config:
    # construct the phylo tree
    if phylo_config.phyloDistances_string == 'None':
        if '031' in run_path: # this run uses a different phylogeny file that had an extra root node which is a mistake
            root = construct_phylo_tree('/home/harishbabu/data/phlyogenyCUB/18Species-with-extra-root-node/1_tree-consensus-Hacket-18Species-modified_cub-names_v1.phy')
        else:
            root = construct_phylo_tree(phylo_config.phylogeny_path)
        print('-'*25 + ' No discretization ' + '-'*25)
    else:
        root = construct_discretized_phylo_tree(phylo_config.phylogeny_path, phylo_config.phyloDistances_string)
        print('-'*25 + ' Discretized ' + '-'*25)
else:
    # construct the tree (original hierarchy as described in the paper)
    root = Node("root")
    root.add_children(['animal','vehicle','everyday_object','weapon','scuba_diver'])
    root.add_children_to('animal',['non_primate','primate'])
    root.add_children_to('non_primate',['African_elephant','giant_panda','lion'])
    root.add_children_to('primate',['capuchin','gibbon','orangutan'])
    root.add_children_to('vehicle',['ambulance','pickup','sports_car'])
    root.add_children_to('everyday_object',['laptop','sandal','wine_bottle'])
    root.add_children_to('weapon',['assault_rifle','rifle'])
    # flat root
    # root.add_children(['scuba_diver','African_elephant','giant_panda','lion','capuchin','gibbon','orangutan','ambulance','pickup','sports_car','laptop','sandal','wine_bottle','assault_rifle','rifle'])
root.assign_all_descendents()

exp_no = int(os.path.basename(run_path)[:3])

if exp_no < 77:
    if ('num_protos_per_descendant' in args) and (args.num_protos_per_descendant > 0):
        for node in root.nodes_with_children():
            node.set_num_protos(args.num_protos_per_descendant)
if exp_no == 77:
    # update num of protos per node based on num_protos_per_descendant
    if args.num_features == 0 and args.num_protos_per_descendant == 0:
        raise Exception('Either of num_features or num_protos_per_descendant must be greater than zero')
    for node in root.nodes_with_children():
        node.set_num_protos(num_protos_per_descendant=args.num_protos_per_descendant,\
                                                            min_protos=args.num_features)

elif 'num_protos_per_child' in args:
    if ('num_protos_per_descendant' in args):
        # update num of protos per node based on num_protos_per_descendant
        if args.num_features == 0 and args.num_protos_per_descendant == 0 and args.num_protos_per_child == 0:
            raise Exception('Either of num_features or num_protos_per_descendant must be greater than zero')
        for node in root.nodes_with_children():
            node.set_num_protos(num_protos_per_descendant=args.num_protos_per_descendant,\
                                num_protos_per_child=args.num_protos_per_child,\
                                min_protos=args.num_features,\
                                split_protos=('protopool' in args) and (args.protopool == 'n'))
else:
    if ('num_protos_per_descendant' in args):
        # update num of protos per node based on num_protos_per_descendant
        if args.num_features == 0 and args.num_protos_per_descendant == 0:
            raise Exception('Either of num_features or num_protos_per_descendant must be greater than zero')
        for node in root.nodes_with_children():
            node.set_num_protos(num_protos_per_descendant=args.num_protos_per_descendant,\
                                min_protos=args.num_features,\
                                split_protos=('protopool' in args) and (args.protopool == 'n'))

------------------------- No discretization -------------------------


# Load model

In [9]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    device_ids = [torch.cuda.current_device()]
else:
    device = torch.device('cpu')
    device_ids = []

# args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
# args = pickle.load(args_file)

# ckpt_file_name = 'net_overspecific_pruned_replaced_thresh=0.5_last'
# ckpt_file_name = 'net_trained_last'
# ckpt_file_name = 'net_trained_10'
# ckpt_file_name = 'net_pretrained'
epoch = ckpt_file_name.split('_')[-1]

ckpt_path = os.path.join(run_path, 'checkpoints', ckpt_file_name)
checkpoint = torch.load(ckpt_path, map_location=device)

if ckpt_file_name != 'net_trained_last':
    print('\n', (10*'-')+'WARNING: Not using the final trained model'+(10*'-'), '\n')

# Obtain the dataset and dataloaders
temp = args.leave_out_classes
args.leave_out_classes = '' # NOTE: because here we need to load the entire dataloader, and filter out the classes later
trainloader, trainloader_pretraining, trainloader_normal, trainloader_normal_augment, projectloader, testloader, test_projectloader, classes = get_dataloaders(args, device)
args.leave_out_classes = temp

print(args.batch_size, trainloader.batch_size)

if len(classes)<=20:
    if args.validation_size == 0.:
        print("Classes: ", testloader.dataset.class_to_idx, flush=True)
    else:
        print("Classes: ", str(classes), flush=True)

# Create a convolutional network based on arguments and add 1x1 conv layer
feature_net, add_on_layers, pool_layer, classification_layers, num_prototypes = get_network(len(classes), args, root=root)
   
# Create a PIP-Net
net = PIPNet(num_classes=len(classes),
                    num_prototypes=num_prototypes,
                    feature_net = feature_net,
                    args = args,
                    add_on_layers = add_on_layers,
                    pool_layer = pool_layer,
                    classification_layers = classification_layers,
                    num_parent_nodes = len(root.nodes_with_children()),
                    root = root
                    )
net = net.to(device=device)
net = nn.DataParallel(net, device_ids = device_ids)    
net.load_state_dict(checkpoint['model_state_dict'],strict=True)
# print(net.eval())
criterion = nn.NLLLoss(reduction='mean').to(device)

# Forward one batch through the backbone to get the latent output size
with torch.no_grad():
    xs1, _, _ = next(iter(trainloader))
    xs1 = xs1.to(device)
    _, proto_features, _, _ = net(xs1)
    wshape = proto_features['root'].shape[-1]
    args.wshape = wshape #needed for calculating image patch size
    print("Output shape: ", proto_features['root'].shape, flush=True)
    
print(args.wshape)


 ----------WARNING: Not using the final trained model---------- 

Num classes (k) =  190 ['cub_001_Black_footed_Albatross', 'cub_002_Laysan_Albatross', 'cub_003_Sooty_Albatross', 'cub_004_Groove_billed_Ani', 'cub_005_Crested_Auklet'] etc.
228 228
Number of prototypes:  768
----------Prototypes per descendant: 1----------
Assigned 196 protos to node root
Assigned 192 protos to node 129+024+067
Assigned 20 protos to node 089+046
Assigned 164 protos to node 129+065
Assigned 24 protos to node 024+051
Assigned 20 protos to node 067+070
Assigned 20 protos to node 089+090
Assigned 20 protos to node 046+087
Assigned 140 protos to node 129+192
Assigned 26 protos to node 065+006
Assigned 20 protos to node 024+031
Assigned 20 protos to node 051+052
Assigned 20 protos to node 067+068
Assigned 128 protos to node 129+043
Assigned 20 protos to node 192+081
Assigned 20 protos to node 065+144
Assigned 20 protos to node 006+071
Assigned 20 protos to node 024+086
Assigned 20 protos to node 031+004
Assig

# Replace tree with tree modified for LOU

In [10]:
# from util.phylogeny import PhylogenyCUB
# from ete3 import Tree, PhyloTree, TreeNode
# format_ = 1

# class LOUPhylogenyCUB(PhylogenyCUB):

#     # Phylogeny class for CUB dataset
#     def __init__(self, filePath, leave_out_classes, node_ids=None, verbose=False):
#         self.node_ids = node_ids
#         self.treeFileNameAndPath = filePath
#         self.total_distance = -1 # -1 means we never calculated it before.

#         self.distance_matrix = {}
#         self.species_groups_within_relative_distance = {}

#         self.get_tree(self.treeFileNameAndPath, leave_out_classes)
#         self.get_total_distance()

#     def get_tree(self, treeFileNameAndPath, leave_out_classes):
#         self.tree = PhyloTree(treeFileNameAndPath, format=format_)
#         self.replace_node(self.tree, leave_out_classes)
#         # setting a dummy name to the internal nodes if it is unnamed
#         for i, node in enumerate(self.tree.traverse("postorder")):
#             if not len(node.name) > 0:
#                 node.name = "temp_" + str(i)

#     def replace_node(self, tree, leave_out_classes):
#         for node in tree.traverse("levelorder"):
#             if not node.is_leaf():
#                 for child in node.children:
#                     if child.is_leaf() and child.name in leave_out_classes:
#                         grandparent = node.up
#                         if grandparent:  # If the node to be replaced is not the root
#                             # grandparent.add_child(child)
#                             # node.detach()

#                             node_idx = grandparent.children.index(node)
#                             idx_to_parent = {grandparent.children.index(parent_node): parent_node for parent_node in grandparent.children}
#                             for parent_node in grandparent.get_children():
#                                 parent_node.detach()
#                             for idx in range(len(idx_to_parent)):
#                                 if idx == node_idx:
#                                     grandparent.add_child(child)
#                                 else:
#                                     grandparent.add_child(idx_to_parent[idx])
#                         else:  # This code does not handle replacing the root because it requires redefining the tree object
#                             raise Exception('Attempt to replace root node')
#                         break  # Stop checking other children since replacement is done

# def construct_phylo_tree_for_LOU(phylogeny_path, leave_out_classes):
#     phylo = LOUPhylogenyCUB(phylogeny_path, leave_out_classes) # '/home/harishbabu/data/phlyogenyCUB'
#     root = Node("root")

#     def set_names_to_internal_nodes(node: TreeNode):
#         """
#         Assumes internal node names are empty strings
#         Assumes leaf node names are in the format 'cub_122_Harris_Sparrow'
#         """
#         if not node.is_leaf():
#             child_names = [set_names_to_internal_nodes(child) for child in node.get_children()]
#             name = "+".join([name.split("+")[0] for name in child_names])
#             if (len(node.get_children()) > 1):
#                 node.name = name
#             return name
#         else:
#             return node.name[4:7] # assumes cub name in the format cub_122_Harris_Sparrow

#     def build_tree(parentnode: Node, ete3_node: TreeNode):
        
#         if ete3_node.is_leaf() or (len(ete3_node.get_children()) > 1):
#             parentnode.add_children(ete3_node.name)
            
#         if (len(ete3_node.get_children()) == 1):
#             build_tree(parentnode, ete3_node.get_children()[0])
            
#         if not ete3_node.is_leaf() and (len(ete3_node.get_children()) > 1):
#             node = parentnode.get_child(ete3_node.name)
#             for ete3_child in ete3_node.get_children():
#                 build_tree(node, ete3_child)

#     set_names_to_internal_nodes(phylo.tree.get_tree_root())

#     for ete3_child in phylo.tree.get_tree_root().get_children():
#         build_tree(root, ete3_child)
        
#     return root

# # if ('leave_out_classes' in args) and (args.leave_out_classes != ''):
# #         with open(args.leave_out_classes, 'r') as file:
# #             leave_out_classes = [line.strip() for line in file]

# modified_root = construct_phylo_tree_for_LOU(phylo_config.phylogeny_path, leave_out_classes)
# modified_root.assign_all_descendents()

# def get_nodes_level(root):
#     active_nodes = []
#     active_nodes += [root]
#     level = 0
#     node_to_level = {}
#     while len(active_nodes) > 0:
#         for node in active_nodes:
#             node_to_level[node.name] = level
#         new_active_nodes = [] 
#         for node in active_nodes:
#             new_active_nodes += node.children
#         active_nodes = new_active_nodes
#         level += 1
#     return node_to_level


# corresponding_node_for_modified_node = {}

# modified_nodes_to_level = get_nodes_level(modified_root)
# nodes_to_level = get_nodes_level(root)

# for modified_node in modified_root.nodes_with_children():
#     corresponding_node_for_modified_node[modified_node.name] = None
#     for node in root.nodes_with_children():
#         if (nodes_to_level[node.name] == modified_nodes_to_level[modified_node.name]) \
#             and (modified_node.leaf_descendents.issubset(node.leaf_descendents)):
#             corresponding_node_for_modified_node[modified_node.name] = node.name

# for modified_node_name, node_name in corresponding_node_for_modified_node.items():
#     modified_node = modified_root.get_node(modified_node_name)
#     modified_node.name = node_name


# def update_children_to_labels(root):
#     for node in root.nodes_with_children():
#         children_to_labels = {}
#         for class_idx, child in enumerate(node.children):
#             children_to_labels[child.name] = class_idx
#         node.children_to_labels = children_to_labels
            
# modified_root.assign_all_descendents()
# update_children_to_labels(modified_root)

# net.module.root = modified_root


In [11]:
# corresponding_node_for_modified_node

In [12]:
set(['1', '2', '3', '4']).issubset(set(['1', '2', '3']))

False

In [13]:
import wandb
from util.log import Log
os.environ['WANDB_DISABLED'] = 'true'
wandb_run = wandb.init(project="pipnet", name=os.path.basename(args.log_dir), config=vars(args), reinit=False)
log = Log(args.log_dir)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [14]:
optimizer_net, optimizer_classifier, params_to_freeze, params_to_train, params_backbone = get_optimizer_nn(net, args)            
scheduler_net = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_net, T_max=len(trainloader)*args.epochs, eta_min=args.lr_net/100.)
if args.epochs<=30:
    scheduler_classifier = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer_classifier, T_0=5, eta_min=0.001, T_mult=1, verbose=False)
else:
    scheduler_classifier = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer_classifier, T_0=10, eta_min=0.001, T_mult=1, verbose=False)

    
if args.OOD_dataset:
    trainloader_OOD, trainloader_pretraining_OOD, trainloader_normal_OOD, trainloader_normal_augment_OOD, projectloader_OOD, testloader_OOD, test_projectloader_OOD, _ = get_dataloaders(args, device, OOD=True)
    print('-'*25 + 'Using OOD data' + '-'*25)
else:
    trainloader_OOD = trainloader_pretraining_OOD = trainloader_normal_OOD = trainloader_normal_augment_OOD = projectloader_OOD = testloader_OOD = test_projectloader_OOD = None
    print('-'*25 + 'Not using OOD data' + '-'*25)
    
if ('focal_loss' in args) and (args.focal_loss == 'y'):
    from util.custom_losses import WeightedCrossEntropyLoss, WeightedNLLLoss, FocalLossWrapper
    criterion = FocalLossWrapper(device=device, alpha=1, gamma=args.focal_loss_gamma, reduction='mean').to(device)
else:
    from util.custom_losses import WeightedCrossEntropyLoss, WeightedNLLLoss, FocalLossWrapper
    criterion = WeightedNLLLoss(device=device).to(device)

chosen network is convnext
-------------------------Not using OOD data-------------------------


In [15]:
def get_heatmap(latent_activation, input_image):
    image_a = latent_activation.cpu().numpy()
    image_a = (image_a - image_a.min()) / (image_a.max() - image_a.min())

    input_image = (input_image - input_image.min()) / (input_image.max() - input_image.min())
    image_b = input_image.permute(1, 2, 0).cpu().numpy()
    
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((input_image.shape[-1], input_image.shape[-1])))
    normalized_heatmap = (reshaped_image_a - np.min(reshaped_image_a)) / (np.max(reshaped_image_a) - np.min(reshaped_image_a))
    
    heatmap_colormap = plt.get_cmap('jet')
    heatmap_colored = heatmap_colormap(normalized_heatmap)
    
    heatmap_colored_uint8 = (heatmap_colored[:, :, :3] * 255).astype(np.uint8)
    image_a_heatmap_pillow = Image.fromarray(heatmap_colored_uint8)
    image_b_pillow = Image.fromarray((image_b * 255).astype('uint8'))
    
    result_image = Image.blend(image_b_pillow, image_a_heatmap_pillow, alpha=0.3)
    
    return np.array(result_image)


def get_heatmap_uninterpolated(latent_activation, input_image):
    image_a = latent_activation.cpu().numpy()
    image_a = (image_a - image_a.min()) / (image_a.max() - image_a.min())

    input_image = (input_image - input_image.min()) / (input_image.max() - input_image.min())
    image_b = input_image.permute(1, 2, 0).cpu().numpy()
    
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((input_image.shape[-1], input_image.shape[-1]), \
                                                                                          resample=Image.NEAREST ))
    normalized_heatmap = (reshaped_image_a - np.min(reshaped_image_a)) / (np.max(reshaped_image_a) - np.min(reshaped_image_a))
    
    heatmap_colormap = plt.get_cmap('jet')
    heatmap_colored = heatmap_colormap(normalized_heatmap)
    
    heatmap_colored_uint8 = (heatmap_colored[:, :, :3] * 255).astype(np.uint8)
    image_a_heatmap_pillow = Image.fromarray(heatmap_colored_uint8)
    image_b_pillow = Image.fromarray((image_b * 255).astype('uint8'))
    
    result_image = Image.blend(image_b_pillow, image_a_heatmap_pillow, alpha=0.3)
    
    return np.array(result_image)

def get_bb_gaussian_threshold(latent_activation, sigma=1.0, percentile=97, extend_h=0, extend_w=0):
    # latent_activation -> []
    upscaled_similarity = get_upscaled_activation_uninterpolated(latent_activation, \
                                                                 image_size=(args.image_size, args.image_size))
    upscaled_similarity = minmaxscale(upscaled_similarity)
    upscaled_similarity = gaussian(upscaled_similarity, sigma=sigma)
    upscaled_similarity = threshold_local(upscaled_similarity, block_size=15, method='mean')
    h_min, h_max, w_min, w_max = find_top_percentile_bbox(upscaled_similarity ,percentile=97)
    h_min = max(0, h_min-extend_h)
    h_max = min(upscaled_similarity.shape[0], h_max+extend_h)
    w_min = max(0, w_min-extend_w)
    w_max = min(upscaled_similarity.shape[1], w_max+extend_w)
    return h_min, h_max, w_min, w_max


def minmaxscale(tensor):
    return (tensor - tensor.min()) / (tensor.max() - tensor.min())

from torch.utils.data import DataLoader, SequentialSampler
from torch.utils.data import DataLoader

def unshuffle_dataloader(dataloader):
    if type(dataloader.dataset) == ImageFolder:
        dataset = dataloader.dataset
    else:
        dataset = dataloader.dataset.dataset.dataset
    new_dataloader = DataLoader(
        dataset=dataset,
        batch_size=dataloader.batch_size,
        shuffle=False,
        num_workers=dataloader.num_workers,
        pin_memory=dataloader.pin_memory,
        drop_last=dataloader.drop_last,
        timeout=dataloader.timeout,
        worker_init_fn=dataloader.worker_init_fn,
        multiprocessing_context=dataloader.multiprocessing_context,
        generator=dataloader.generator,
        prefetch_factor=dataloader.prefetch_factor,
        persistent_workers=dataloader.persistent_workers
    )
    return new_dataloader

# Remove classes from dataloader

In [16]:
from torch.utils.data import Sampler, SubsetRandomSampler

def create_filtered_dataloader(dataloader, new_sampler, batch_size):
    if type(dataloader.dataset) == ImageFolder:
        dataset = dataloader.dataset
    else:
        dataset = dataloader.dataset.dataset.dataset
    new_dataloader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=False,
        sampler=new_sampler,
        num_workers=dataloader.num_workers,
        pin_memory=dataloader.pin_memory,
        drop_last=dataloader.drop_last,
        timeout=dataloader.timeout,
        worker_init_fn=dataloader.worker_init_fn,
        multiprocessing_context=dataloader.multiprocessing_context,
        generator=dataloader.generator,
        prefetch_factor=dataloader.prefetch_factor,
        persistent_workers=dataloader.persistent_workers
    )
    return new_dataloader

leave_out_loader = testloader
# leave_out_loader = test_projectloader
unique_labels = set()
total_samples = 0
for xs, ys in testloader:
    unique_labels.update(ys.tolist())
    total_samples += xs.shape[0]
print("testloader Unique Labels:", unique_labels)
print("testloader total_samples:", total_samples)

# leave_out_classes = args.leave_out_classes.split(',')[2]

# class_of_interest = 0
# classes_to_keep = [leave_out_classes[class_of_interest]]

# classes_to_keep = [leave_out_classes[1], leave_out_classes[6], leave_out_classes[8]]
# classes_to_keep = [leave_out_classes[0], leave_out_classes[1], leave_out_classes[2], leave_out_classes[3], leave_out_classes[6], leave_out_classes[7]]


classes_to_keep = leave_out_classes

# leave_out_classes = ['cub_093_Clark_Nutcracker']

print('classes_to_keep', classes_to_keep)
idx_of_classes_to_keep = set()
name2label = leave_out_loader.dataset.class_to_idx # param
label2name = {label:name for name, label in name2label.items()}
for label in label2name:
    # NOTE: Keeping the left out classes here
    if label2name[label] in classes_to_keep:
        idx_of_classes_to_keep.add(label)

target_indices = []
for i in range(len(leave_out_loader.dataset)):
    *_, label = leave_out_loader.dataset[i]
    if label in idx_of_classes_to_keep:
        target_indices.append(i)
sampler = SubsetRandomSampler(target_indices)
to_shuffle = False
    
leave_out_loader = create_filtered_dataloader(leave_out_loader, sampler, batch_size=256) # dataloader.batch_size
unique_labels = set()
total_samples = 0
for xs, ys in leave_out_loader:
    unique_labels.update(ys.tolist())
    total_samples += xs.shape[0]
print("leave_out_loader Unique Labels:", unique_labels)
print("leave_out_loader total_samples:", total_samples)

name2label = leave_out_loader.dataset.class_to_idx
label2name = {label:name for name, label in name2label.items()}

# print('Leave out classes', args.leave_out_classes)

testloader Unique Labels: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189}
testloader total_samples: 5512
classes_to_keep ['cub_149_Brown_Thrasher', 'cub_030_Fish_Crow', 'cub_191_Red_headed_Woodpecker', 'cub_

In [17]:
leave_out_loader.dataset.class_to_idx

{'cub_001_Black_footed_Albatross': 0,
 'cub_002_Laysan_Albatross': 1,
 'cub_003_Sooty_Albatross': 2,
 'cub_004_Groove_billed_Ani': 3,
 'cub_005_Crested_Auklet': 4,
 'cub_006_Least_Auklet': 5,
 'cub_007_Parakeet_Auklet': 6,
 'cub_008_Rhinoceros_Auklet': 7,
 'cub_009_Brewer_Blackbird': 8,
 'cub_010_Red_winged_Blackbird': 9,
 'cub_011_Rusty_Blackbird': 10,
 'cub_012_Yellow_headed_Blackbird': 11,
 'cub_013_Bobolink': 12,
 'cub_014_Indigo_Bunting': 13,
 'cub_015_Lazuli_Bunting': 14,
 'cub_016_Painted_Bunting': 15,
 'cub_017_Cardinal': 16,
 'cub_018_Spotted_Catbird': 17,
 'cub_019_Gray_Catbird': 18,
 'cub_020_Yellow_breasted_Chat': 19,
 'cub_021_Eastern_Towhee': 20,
 'cub_023_Brandt_Cormorant': 21,
 'cub_024_Red_faced_Cormorant': 22,
 'cub_025_Pelagic_Cormorant': 23,
 'cub_026_Bronzed_Cowbird': 24,
 'cub_027_Shiny_Cowbird': 25,
 'cub_028_Brown_Creeper': 26,
 'cub_029_American_Crow': 27,
 'cub_030_Fish_Crow': 28,
 'cub_031_Black_billed_Cuckoo': 29,
 'cub_032_Mangrove_Cuckoo': 30,
 'cub_033_Ye

In [18]:
if 'cl_weight' not in args:
    args.cl_weight = 2.0

# Leave out without mask pruning

In [20]:
# testloader

test_info, log_dict = test_pipnet(net, leave_out_loader, optimizer_net, optimizer_classifier, \
                                    scheduler_net, scheduler_classifier, criterion, 0, \
                                        args.epochs, device, pretrain=False, finetune=False, \
                                        test_loader_OOD=testloader_OOD, kernel_orth=args.kernel_orth == 'y', \
                                            tanh_desc=args.tanh_desc == 'y', align=args.align == 'y', uni=args.uni == 'y', align_pf=args.align_pf == 'y',\
                                            minmaximize=args.minmaximize == 'y', wandb_run=wandb_run, pretrain_epochs=args.epochs_pretrain, log=log, \
                                          args=args, apply_overspecificity_mask=False, leave_out_classes=leave_out_classes, path_prob_softmax_tau=1)

Test Epoch0: 100% 2/2 [00:03<00:00,  1.55s/it, L:0.949,LC:1.166, L_OVSP:-0.006, L_MASKL1:0.002, LA_PF:0.019, LT:-5.000, L_ORTH:0.002, LT_DESC:-5.000, L_OOD_ENT:0.562, losses_used:MASK_PRUNING+MIN_CONT+AL_PF+KO+CL+OOD_ENT]


	Fine accuracy: 0.52
	Node name: root, acc: 99.31, f1:99.65, samples: 580, 129+024+067=576/580=0.99, 089+046=0/0=nan
	Node name: 129+024+067, acc: 84.14, f1:84.11, samples: 580, 129+065=388/418=0.93, 024+051=100/162=0.62, 067+070=0/0=nan
	Node name: 089+046, acc: inf, f1:inf, samples: 0, 089+090=0/0=nan, 046+087=0/0=nan
	Node name: 129+065, acc: 98.56, f1:98.55, samples: 418, 129+192=298/298=1.0, 065+006=114/120=0.95
	Node name: 024+051, acc: 92.59, f1:92.64, samples: 162, 024+031=94/102=0.92, 051+052=56/60=0.93
	Node name: 067+070, acc: inf, f1:inf, samples: 0, 067+068=0/0=nan, cub_070_Green_Violetear=0/0=nan
	Node name: 089+090, acc: inf, f1:inf, samples: 0, cub_089_Hooded_Merganser=0/0=nan, cub_090_Red_breasted_Merganser=0/0=nan
	Node name: 046+087, acc: inf, f1:inf, samples: 0, cub_046_Gadwall=0/0=nan, cub_087_Mallard=0/0=nan
	Node name: 129+192, acc: 86.58, f1:84.17, samples: 298, 129+043=236/238=0.99, 192+081=22/60=0.37
	Node name: 065+006, acc: 100.0, f1:100.0, samples: 120, 065

In [60]:
# # test_info['node_accuracy'].values()

# # print(leave_out_classes[class_of_interest])

# # for key in test_info['node_accuracy']:
# #     if leave_out_classes[class_of_interest] in modified_root.get_node(key).leaf_descendents:
# #         print(key, test_info['node_accuracy'][key]['accuracy'])

# print(leave_out_classes[class_of_interest])
# leave_out_accuracy = 1
# for key in test_info['node_accuracy']:
#     if leave_out_classes[class_of_interest] in root.get_node(key).leaf_descendents:
#         if not (leave_out_classes[class_of_interest] in [child.name for child in root.get_node(key).children]):
#             leave_out_accuracy *= test_info['node_accuracy'][key]['accuracy'] / 100
#         print(key, test_info['node_accuracy'][key]['accuracy'])

# print(leave_out_accuracy)

In [61]:
# print(test_info.keys())

# test_info['node_accuracy']

In [62]:
# log_dict.keys()



In [63]:
# modified_root.get_node('107+093').children_to_labels
# root.get_node('107+093').children_to_labels


In [64]:
# corresponding_node_for_modified_node['129+024+067']

# modified_root.get_node('129+024+067').children_to_labels
# root.get_node('129+024+067').children_to_labels

# Leave out with mask pruning

In [19]:
# testloader

test_info, log_dict = test_pipnet(net, leave_out_loader, optimizer_net, optimizer_classifier, \
                                    scheduler_net, scheduler_classifier, criterion, 0, \
                                        args.epochs, device, pretrain=False, finetune=False, \
                                        test_loader_OOD=testloader_OOD, kernel_orth=args.kernel_orth == 'y', \
                                            tanh_desc=args.tanh_desc == 'y', align=args.align == 'y', uni=args.uni == 'y', align_pf=args.align_pf == 'y',\
                                            minmaximize=args.minmaximize == 'y', wandb_run=wandb_run, pretrain_epochs=args.epochs_pretrain, log=log, \
                                          args=args, apply_overspecificity_mask=True, leave_out_classes=leave_out_classes, path_prob_softmax_tau=1)

Test Epoch0: 100% 2/2 [00:03<00:00,  1.62s/it, L:1.455,LC:1.571, L_OVSP:-0.006, L_MASKL1:0.002, LA_PF:0.019, LT:-5.000, L_ORTH:0.002, LT_DESC:-5.000, L_OOD_ENT:0.550, losses_used:MASK_PRUNING+MIN_CONT+AL_PF+KO+CL+OOD_ENT]
/home/harishbabu/.conda/envs/hpnet4/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/harishbabu/.conda/envs/hpnet4/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


	Fine accuracy: 0.0
	Node name: root, acc: 0.0, f1:0.0, samples: 580, 129+024+067=0/580=0.0, 089+046=0/0=nan
	Node name: 129+024+067, acc: 24.48, f1:11.8, samples: 580, 129+065=0/418=0.0, 024+051=142/162=0.88, 067+070=0/0=nan
	Node name: 089+046, acc: inf, f1:inf, samples: 0, 089+090=0/0=nan, 046+087=0/0=nan
	Node name: 129+065, acc: 71.29, f1:59.34, samples: 418, 129+192=298/298=1.0, 065+006=0/120=0.0
	Node name: 024+051, acc: 37.04, f1:20.02, samples: 162, 024+031=0/102=0.0, 051+052=60/60=1.0
	Node name: 067+070, acc: inf, f1:inf, samples: 0, 067+068=0/0=nan, cub_070_Green_Violetear=0/0=nan
	Node name: 089+090, acc: inf, f1:inf, samples: 0, cub_089_Hooded_Merganser=0/0=nan, cub_090_Red_breasted_Merganser=0/0=nan
	Node name: 046+087, acc: inf, f1:inf, samples: 0, cub_046_Gadwall=0/0=nan, cub_087_Mallard=0/0=nan
	Node name: 129+192, acc: 20.13, f1:6.75, samples: 298, 129+043=0/238=0.0, 192+081=60/60=1.0
	Node name: 065+006, acc: 91.67, f1:95.65, samples: 120, 065+144=110/120=0.92, 006+

In [55]:
# print(net.module._root_proto_presence)
# print(net.module._root_proto_presence.shape)

In [56]:
# net.module._root_proto_presence[:, 0].shape

In [57]:
# torch.cat([net.module._root_proto_presence[:, 1].unsqueeze(-1), net.module._root_proto_presence[:, 0].unsqueeze(-1)], dim=-1)#.shape

In [25]:
# with torch.no_grad():
#     for node in root.nodes_with_children():
#         net.module._root_proto_presence
#         proto_presence = getattr(net.module, '_'+node.name+'_proto_presence')
#         reversed_proto_presence = torch.cat([proto_presence[:, 1].unsqueeze(-1), proto_presence[:, 0].unsqueeze(-1)], dim=-1)
#         setattr(net.module, '_'+node.name+'_proto_presence', \
#                 nn.Parameter(reversed_proto_presence))

#     test_info, log_dict = test_pipnet(net, leave_out_loader, optimizer_net, optimizer_classifier, \
#                                     scheduler_net, scheduler_classifier, criterion, 0, \
#                                         args.epochs, device, pretrain=False, finetune=False, \
#                                         test_loader_OOD=testloader_OOD, kernel_orth=args.kernel_orth == 'y', \
#                                             tanh_desc=args.tanh_desc == 'y', align=args.align == 'y', uni=args.uni == 'y', align_pf=args.align_pf == 'y',\
#                                             minmaximize=args.minmaximize == 'y', wandb_run=wandb_run, pretrain_epochs=args.epochs_pretrain, log=log, \
#                                           args=args, apply_overspecificity_mask=True)

In [19]:
torch.tensor([False, True, True]).sum()

tensor(2)